In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00


In [2]:
# 设置OpenAI KEY环境变量
import os
import getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


假设我们有一个公司产品相关的数据库，我们可以定义一个函数来对这个数据库进行搜索

In [3]:
from openai import OpenAI

client = OpenAI()
import json

# 示例函数
def find_product(sql_query):
    # Execute query here
    results = [
        {"name": "pen", "color": "blue", "price": 1.99},
        {"name": "pen", "color": "red", "price": 1.78},
    ]
    return results

# 函数定义
function_find_product = {
        "name": "find_product",
        "description": "Get a list of products from a sql query",
        "parameters": {
            "type": "object",
            "properties": {
                "sql_query": {
                    "type": "string",
                    "description": "A SQL query",
                }
            },
            "required": ["sql_query"],
        },
    }



def run(user_question):
    # 传入用户question和函数给GPT3.5
    messages = [{"role": "user", "content": user_question}]

    response = client.chat.completions.create(model="gpt-3.5-turbo-0613", messages=messages, tools=[{"type": "function", "function": function_find_product }])
    response_message = response.choices[0].message

    # 添加assistant响应到messages中
    messages.append(response_message)


    # 调用函数
    function_name = response_message.tool_calls[0].function.name
    if function_name == "find_product":
        function_args = json.loads(
            response_message.tool_calls[0].function.arguments
        )
        products = find_product(function_args.get("sql_query"))
    else:
        # Handle error
        products = []
    # 把函数响应结果添加到messages中
    messages.append(
        {
            "role": "tool",
            "content": json.dumps(products),
            "tool_call_id": response_message.tool_calls[0].id,
        }
    )
    # 把函数响应结果转换为自然语言
    second_response = client.chat.completions.create(model="gpt-3.5-turbo-0613",
    messages=messages)
    return second_response.choices[0].message.content


print(run("I need the top 2 products where the price is less than 2.00"))


The top 2 products with prices less than 2.00 are:

1. Pen (color: blue, price: 1.99)
2. Pen (color: red, price: 1.78)
